<a href="https://colab.research.google.com/github/Magdy007/AIMusician/blob/main/GEPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import subprocess
import torch

# التأكد من تفعيل الـ GPU في كولاب
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ الجهاز المستخدم: {device}")

# تثبيت المكتبات المطلوبة في Google Colab
try:
    import google.colab
    !pip install -q transformers spacy gradio
except ImportError:
    pass  # التخطي إذا لم يكن يعمل في كولاب

import spacy
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# تحميل نموذج Spacy إذا لم يكن محملاً مسبقًا
subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"], check=True)
nlp = spacy.load("en_core_web_sm")

# تحميل النموذج اللغوي من Hugging Face مع دعم GPU
model_name = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# تجهيز وظيفة توليد النصوص باستخدام Transformers
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)

# دالة معالجة النصوص
def chatbot_response(message):
    response = text_generator(message, max_length=100)
    return response[0]['generated_text']

# إنشاء واجهة تفاعلية باستخدام Gradio
gr.Interface(fn=chatbot_response, inputs="text", outputs="text").launch(share=True)


✅ الجهاز المستخدم: cpu


Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a8294c557ea12c52f0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
import os
import subprocess
import torch
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# التأكد من تفعيل الـ GPU في كولاب
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ الجهاز المستخدم: {device}")

# تثبيت المكتبات المطلوبة في Google Colab
try:
    import google.colab
    !pip install -q transformers spacy gradio
except ImportError:
    pass  # التخطي إذا لم يكن يعمل في كولاب

import spacy

# تحميل نموذج Spacy إذا لم يكن محملاً مسبقًا
subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"], check=True)
nlp = spacy.load("en_core_web_sm")

# تحميل نموذج أكثر دقة من Hugging Face مع دعم GPU
model_name = "mistralai/Mistral-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

# تجهيز وظيفة توليد النصوص باستخدام Transformers
text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device == "cuda" else -1)

# دالة معالجة النصوص مع تحسين جودة الردود
def chatbot_response(message):
    response = text_generator(message, max_length=100, do_sample=True, temperature=0.7, top_k=50)
    return response[0]['generated_text']

# إنشاء واجهة تفاعلية باستخدام Gradio بحجم أصغر وزر إعدادات واضح
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 Chatbot - مساعد الذكاء الاصطناعي")
    chatbot = gr.Chatbot(height=300)
    user_input = gr.Textbox(placeholder="اكتب رسالتك هنا...", lines=2)
    submit_btn = gr.Button("إرسال")
    close_btn = gr.Button("إغلاق")
    settings_btn = gr.Button("⚙️ الإعدادات", elem_id="settings-btn")

    def respond(message, chat_history):
        response = chatbot_response(message)
        chat_history.append((message, response))
        return chat_history

    submit_btn.click(respond, [user_input, chatbot], chatbot)
    close_btn.click(lambda: demo.close(), None, None)

demo.launch(share=True)


✅ الجهاز المستخدم: cpu


OSError: mistralai/Mistral-7B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`